In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import math

import time
import pandas as pd
import folium as fl
import requests
import geopandas as gpd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable, GeocoderQuotaExceeded
from geopy.extra.rate_limiter import RateLimiter
import re

In [156]:
houses = pd.read_csv("Export/Propriétés_map.csv")
Communities = pd.read_csv("Export/Communitycenters.csv")
Mairies = pd.read_csv("Export/Mairies.csv")
Parcs = pd.read_csv("Export/Parcs.csv")
Sportscenters = pd.read_csv("Export/Sportscenters.csv")
Demo = pd.read_csv("CSVs/TEMPLATE.csv")
Superficie = pd.read_csv("CSVs/Superficie_quartier.csv")
houses

for i,r in houses.iterrows():
    if pd.isna(r["NOM"]):
        print(r["Addresse"])
    #df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)




5076


In [51]:
test = houses.groupby("NOM").mean().round(2)

quartier_liste = [i for i,r in test.iterrows()]
my_dict = {i:r[:0-3] for i,r in test.iterrows()}


car_score           6.68
cafe_score          6.60
walk_score          8.77
park_score          8.50
quiet_score         6.32
cycling_score       4.59
transit_score       8.65
vibrant_score       5.12
daycares_score      8.56
primary_score       9.30
shopping_score      8.87
groceries_score     7.27
nightlife_score     4.85
restaurant_score    7.85
Name: Westmount, dtype: float64
['Ahuntsic-Cartierville', 'Anjou', "Baie-d'Urfé", 'Beaconsfield', 'Côte-Saint-Luc', 'Côte-des-Neiges-Notre-Dame-de-Grâce', 'Dollard-des-Ormeaux', 'Dorval', 'Hampstead', 'Kirkland', "L'Île-Bizard-Sainte-Geneviève", 'LaSalle', 'Lachine', 'Le Plateau-Mont-Royal', 'Le Sud-Ouest', 'Mercier-Hochelaga-Maisonneuve', 'Montréal-Est', 'Montréal-Nord', 'Montréal-Ouest', 'Outremont', 'Pierrefonds-Roxboro', 'Pointe-Claire', 'Rivière-des-Prairies-Pointe-aux-Trembles', 'Rosemont-La Petite-Patrie', 'Saint-Laurent', 'Saint-Léonard', 'Sainte-Anne-de-Bellevue', 'Verdun', 'Ville-Marie', 'Villeray-Saint-Michel-Parc-Extension', 'West

In [50]:
café = int(input("How often often do you drink coffee per week?"))
car = int(input("How often often do you drink coffee per week?"))
quiet = int(input("How often often do you drink coffee per week?"))
Total_score = []
Quartier_liste = []
Recommandations = {}
for k,v in my_dict.items():
    Quartier_liste.append(k)
café_score = []
car_score = []
quiet_score = []
for i in quartier_liste:
    café_score.append(café*my_dict[i][1])
    car_score.append(car*my_dict[i][0])
    quiet_score.append(quiet*my_dict[i][4])
    Total_score.append(round(café*my_dict[i][1]+car*my_dict[i][0]+quiet*my_dict[i][4],0))

Recommandations = {Quartier_liste[i]: Total_score[i] for i in range(len(Total_score))}
print(Recommandations)

{'Ahuntsic-Cartierville': 57.0, 'Anjou': 61.0, "Baie-d'Urfé": 50.0, 'Beaconsfield': 52.0, 'Côte-Saint-Luc': 54.0, 'Côte-des-Neiges-Notre-Dame-de-Grâce': 55.0, 'Dollard-des-Ormeaux': 52.0, 'Dorval': 60.0, 'Hampstead': 54.0, 'Kirkland': 68.0, "L'Île-Bizard-Sainte-Geneviève": 60.0, 'LaSalle': 59.0, 'Lachine': 57.0, 'Le Plateau-Mont-Royal': 62.0, 'Le Sud-Ouest': 54.0, 'Mercier-Hochelaga-Maisonneuve': 53.0, 'Montréal-Est': 54.0, 'Montréal-Nord': 52.0, 'Montréal-Ouest': 56.0, 'Outremont': 55.0, 'Pierrefonds-Roxboro': 59.0, 'Pointe-Claire': 62.0, 'Rivière-des-Prairies-Pointe-aux-Trembles': 60.0, 'Rosemont-La Petite-Patrie': 50.0, 'Saint-Laurent': 57.0, 'Saint-Léonard': 56.0, 'Sainte-Anne-de-Bellevue': 39.0, 'Verdun': 61.0, 'Ville-Marie': 53.0, 'Villeray-Saint-Michel-Parc-Extension': 49.0, 'Westmount': 59.0}


In [90]:
import numpy as np

scores = {
    "Ville1": {"restaurant": 40, "parcs": 50},
    "Ville2": {"restaurant": 20, "parcs": 40},
    "Ville3": {"restaurant": 35, "parcs": 90},
    "Ville4": {"restaurant": 50, "parcs": 10}
}

# Standardisation des scores
mean = np.mean([scores[k][c] for k in scores for c in scores[k]])
std = np.std([scores[k][c] for k in scores for c in scores[k]])

for k in scores:
    for c in scores[k]:
        scores[k][c] = (scores[k][c] - mean) / std

# Normalisation des scores sur une échelle de 0 à 10
for c in scores["Ville1"]:
    min_c = min(scores[k][c] for k in scores)
    max_c = max(scores[k][c] for k in scores)
    for k in scores:
        scores[k][c] = (scores[k][c] - min_c) / (max_c - min_c) * 10

print(scores)


{'Ville1': {'restaurant': 6.666666666666668, 'parcs': 4.999999999999999}, 'Ville2': {'restaurant': 0.0, 'parcs': 3.7499999999999996}, 'Ville3': {'restaurant': 5.000000000000001, 'parcs': 10.0}, 'Ville4': {'restaurant': 10.0, 'parcs': 0.0}}


In [94]:
import numpy as np

scores = {
    "Quartier1": {"restaurant": 30, "parc": 20, "ecole": 10, "superficie": 2},
    "Quartier2": {"restaurant": 50, "parc": 40, "ecole": 5, "superficie": 50},
    "Quartier3": {"restaurant": 20, "parc": 10, "ecole": 30, "superficie": 5},
    "Quartier4": {"restaurant": 40, "parc": 30, "ecole": 20, "superficie": 10}
}

# Pondération des scores
weights = {"restaurant": 0.7, "parc": 0.7, "ecole": 0.7, "superficie": 0.3}
critere_scores = {c: {} for c in weights}
for k in scores:
    for c in scores[k]:
        if c != "superficie":
            critere_scores[c][k] = weights[c] * scores[k][c]

# Normalisation des scores pour chaque critère sur une échelle de 0 à 10
for c in weights:
    min_score = min(critere_scores[c].values())
    max_score = max(critere_scores[c].values())
    for k in scores:
        if c in scores[k]:
            scores[k][c + "_score"] = (critere_scores[c][k] - min_score) / (max_score - min_score) * 10

print(scores)



ValueError: min() arg is an empty sequence

In [100]:
data = {
    'Ville1': {'restaurant': 40, 'parcs': 50, 'ecoles': 20, 'superficie': 10},
    'Ville2': {'restaurant': 20, 'parcs': 40, 'ecoles': 30, 'superficie': 5},
    'Ville3': {'restaurant': 35, 'parcs': 60, 'ecoles': 15, 'superficie': 20},
    'Ville4': {'restaurant': 50, 'parcs': 10, 'ecoles': 15, 'superficie': 15}
}

max_values = {}
total_area = 0
for ville, features in data.items():
    total_area += features['superficie']
    for feature, value in features.items():
        if feature != 'superficie':
            max_values[feature] = max(max_values.get(feature, 0), value)

scores = {}
for ville, features in data.items():
    total_score = 0
    for feature, value in features.items():
        if feature == 'superficie':
            total_score += 0.3 * value / total_area
        else:
            score = 0.7 * value / max_values[feature]
            scores[ville] = scores.get(ville, {})
            scores[ville][feature] = score
            total_score += score
    scores[ville]['total_score'] = total_score

normalized_scores = {}
for feature in max_values:
    normalized_scores[feature] = {}
    feature_scores = [scores[ville][feature] for ville in data]
    max_score = max(feature_scores)
    min_score = min(feature_scores)
    for ville, score in scores.items():
        normalized_score = (score[feature] - min_score) / (max_score - min_score) * 10
        normalized_scores[feature][ville] = normalized_score
  #      normalized_score = (score - min_score) / (max_score - min_score) * 10

print(normalized_scores)


TypeError: unsupported operand type(s) for -: 'dict' and 'float'

In [117]:
def normalize_scores(data):
    categories = ["restaurant", "parcs", "ecoles", "superficie"]
    weights = {"restaurant": 0.7, "parcs": 0.7, "ecoles": 0.7, "superficie": 0.3}

    # Calculate the minimum and maximum scores for each category
    min_scores = {category: min(data, key=lambda x: data[x].get(category, float('inf')))[category] for category in categories}
    max_scores = {category: max(data, key=lambda x: data[x].get(category, float('-inf')))[category] for category in categories}

    # Calculate the weighted total score for each area
    for city in data:
        total_score = 0
        for category in categories:
            score = data[city][category]
            normalized_score = (score - min_scores[category]) / (max_scores[category] - min_scores[category])
            total_score += normalized_score * weights[category]
        data[city]["score"] = total_score

    return data
